<a href="https://colab.research.google.com/github/Super-rookie-Py/_WebProgramming_Class/blob/master/%EB%B0%95%EA%B1%B4%EC%9A%B0__CGV_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selenium CGV 영화 리뷰 스크래핑

## Selenium 및 웹 드라이버 설치

In [ ]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

     |████████████████████████████████| 911kB 2.9MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [900 kB]
Get:12 http://a

## CGV 영화 리뷰 긁어오기

* 아이언맨: http://www.cgv.co.kr/movies/detail-view/?midx=38262#1
* 다크나이트: http://www.cgv.co.kr/movies/detail-view/?midx=76417#1
* url을 통해 리뷰 페이지 접근 불가
* 셀레니움으로 페이지 번호를 클릭하여 접근

In [51]:
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

def get_movie_reviews(url, page_num=10):

  wd = webdriver.Chrome('chromedriver', options=chrome_options)
  wd.get(url)

  writer_list = []
  review_list = []
  date_list = []

  for page_no in range(1, page_num+1):
    try:
      page_ul = wd.find_element_by_id('paging_point')
      page_a = page_ul.find_element_by_link_text(str(page_no))
      page_a.click()
      time.sleep(1)

      writers = wd.find_elements_by_class_name('writer-name')
      writer_list += [writer.text for writer in writers]
      reviews = wd.find_elements_by_class_name('box-comment')
      review_list += [review.text for review in reviews]
      dates = wd.find_elements_by_class_name('day')
      date_list += [date.text for date in dates]

      if page_no % 10 == 0:
        next_button = page_ul.find_element_by_class_name('btn-paging.next')
        next_button.click()
        time.sleep(1)

    except NoSuchElementException:
      break

  movie_review_df = pd.DataFrame({'Writer': writer_list,
                                  'Review': review_list,
                                  "Date": date_list})
  
  return movie_review_df

In [52]:
url = 'http://www.cgv.co.kr/movies/detail-view/?midx=83381'
movie_review_df = get_movie_reviews(url,13)
movie_review_df

,Writer,Review,Date
0,bb**g0813,어려워요. 여러번 봐야겠어요.,2020.09.15
1,po**762,약간 이해하기 어려운 부분이 잇지만 너무 재밋어여 2번정도 더보고 싶어요 ㅎㅎ,2020.09.15
2,황여사의 영화데이트,n차 관람을 해도 이해하기 어려울거 같다 인터스텔라도 인셉션도 어느 정도 이해하고 ...,2020.09.15
3,밍구스,어렵어렵..어려웠지만 그래도 영상미도 훌륭하네요,2020.09.15
4,qd**on,보다보면헷갈리고 뇌정지오는데 재밌어요,2020.09.15
...,...,...,...
73,카르페 디엠!,이해하기가 어려웠어요...,2020.09.15
74,유키나라,영상미와 화려항 액션 하지만 절대 이해하지못할 스토리와 배경이다.,2020.09.15
75,써밋컬러,난해하네요? 인류를 위한다는 목적은 좋았으나 영화를 이해함에 어려움이 있고 긴 상영...,2020.09.15
76,어슬펀 영화광,놀란 영화는 아이맥스에서 봐야 멋있다...놀란 영화는 음악이 많은 부분을 차지하는것...,2020.09.15


## CGV 상영작 스크래핑

* http://www.cgv.co.kr/movies/

In [ ]:
url = "http://www.cgv.co.kr/movies/"

wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.get(url)

movie_chart = wd.find_element_by_class_name('sect-movie-chart')
contents = movie_chart.find_elements_by_class_name('box-contents')
for content in contents:
  link = content.find_element_by_tag_name('a').get_attribute('href')
  title = content.find_element_by_class_name('title').text
  percent = content.find_element_by_class_name('percent').text
  info = content.find_element_by_class_name('txt-info').text
  print(title, percent, info, link)
  print(get_movie_reviews(link, 2))

테넷 예매율22.6% 2020.08.26 개봉 http://www.cgv.co.kr/movies/detail-view/?midx=83381
        Writer                                    Review        Date
0      ye**47a                       너무 어려웠지만 액션이 좋았던 영화  2020.09.15
1     wlgmlgml                 약간 내용이 조금 이해가 안 됐지만 재밌었어요  2020.09.15
2   he**he6067                    너무 어렵지만 신선해서 집중도 있는 영화  2020.09.15
3       sh**33                          스토리가 알차고 흥미진진하네요  2020.09.15
4     ql**7747  이해는 잘 못했지만 지루한 틈이 없어서 러닝타임이 길다고 느껴지지 않았음  2020.09.15
5      SoldatJ                             기대이상, 최고였습니다.  2020.09.15
6     ja**viny           인버션이라는 소재 하나를 더했을뿐 전형적인 헐리우드 서사  2020.09.15
7         범이♡♡                  음 무슨 내용인지는 모르겠지만 볼만했다ㅋㅋㅋ  2020.09.15
8    ki**isu26                             시간가는줄 모르고 봤어요  2020.09.15
9        집시레인져                 쉽게 이해가 힘듬 이해가안되면 느껴야 하는건지  2020.09.15
10  mu**el0410                        감독님 만수무강하시고 다작해주세요  2020.09.15
11         와사B                          이해하지말고 느껴라..맞는말!  2020.09.15
오! 문희 예매율6.7% 2020.09.02 